In [ ]:
from lore_sa.dataset import TabularDataset
from lore_sa.lore import TabularGeneticGeneratorLore

In [ ]:
target = 'income'

In [ ]:
dataset = TabularDataset.from_csv('adult.csv', class_name = target)
dataset.df.dropna(inplace = True)

In [ ]:
dataset.df.keys()

In [ ]:
dataset.df.drop(["marital-status", "fnlwgt", "educational-num", "occupation", "native-country"], axis=1, inplace=True)

In [ ]:
dataset.update_descriptor()

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from lore_sa.bbox import sklearn_classifier_bbox

def train_model(dataset: TabularDataset):
    numeric_indices = [v['index'] for v in dataset.descriptor['numeric'].values()]
    categorical_indices = [v['index'] for v in dataset.descriptor['categorical'].values()]
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', StandardScaler(), numeric_indices),
            ('cat', OrdinalEncoder(), categorical_indices)
        ]
    )
    model = make_pipeline(preprocessor, RandomForestClassifier(n_estimators=100, random_state=42))
    
    X = dataset.df.drop([target], axis=1)
    X_train, X_test, y_train, y_test = train_test_split(X.values, dataset.df[target].values,
                test_size=0.3, random_state=42, stratify=dataset.df[target].values)
    model.fit(X_train, y_train)
    
    return sklearn_classifier_bbox.sklearnBBox(model)

In [ ]:
bbox = train_model(dataset)

In [ ]:
tabularLore = TabularGeneticGeneratorLore(bbox, dataset)

In [ ]:
out = []
for k in dataset.descriptor.keys():
    if k != 'target':
        out.extend(list(dataset.descriptor[k].keys()))
out, dataset.descriptor.keys()

In [ ]:
dataset.descriptor["target"]

In [ ]:
x = (dataset.df.drop([target], axis=1)).iloc[1]

In [ ]:
# tabularLore.explain(x)

In [ ]:
for k,v in x.items():
    print (k,v)

In [ ]:
# tabularLore.interactive_explanation(x, inJupyter=False)

In [ ]:
tabularLore.interactive_explanation(inJupyter=False)

In [ ]:
import os
os.environ["INSTANCE_PROVIDED"], os.environ["CUSTOM_DATA_LOADED"]

In [ ]:
from lore_sa.webapp.routes.webapp_api_state import webapp_state

In [ ]:
webapp_state.dataset.df

In [ ]:
from lore_sa.webapp import Webapp
webapp = Webapp()

In [ ]:
webapp.launch_demo()